In [1]:
import gzip
import csv
from IPython.display import clear_output

In [ ]:
with gzip.open('extra.xml.gz', 'rt') as f:
    with open('idr_annotations_no_pfam.csv', 'a') as fOut:
        span = False
        for idx, line in enumerate(f):
            if 'protein id' in line:
                prot = line.split('"')
                fOut.write('\n'+prot[1]+',')
            elif 'match' in line and 'dbname="MOBIDBLT"' in line:
                span = True
            elif '</match>' in line:
                span = False
            elif span:
                bound = line.split('"')
                fOut.write('d;'+bound[1]+':'+bound[3]+'|')
            if idx%1000000==0:
                clear_output()
                print(f'Processing Row: {idx}')

In [2]:
with open('pfam_domains_grouped_len_no_overlap.csv') as f:
    rdr = csv.reader(f)
    pfam_protein_set_1 = set()
    for idx, line in enumerate(rdr):
        if idx > 90000000:
            break
        pfam_protein_set_1.add(line[0])
        if idx % 1000000 == 0:
            clear_output()
            print('row', idx)

row 90000000


In [3]:
with open('idr_annotations_no_pfam.csv') as f:
    with open('idr_annotations_pfam_proteins.csv', 'w') as fOut:
        for idx, line in enumerate(f):
            l = line.split(',')
            if l[0] in pfam_protein_set_1 and l[1][:-1] != '':
                fOut.write(line)
            if idx % 1000000 == 0:
                clear_output()
                print(f'Processing Row {idx}')

Processing Row 230000000


In [4]:
del pfam_protein_set_1

In [5]:
with open('pfam_domains_grouped_len_no_overlap.csv') as f:
    rdr = csv.reader(f)
    pfam_protein_set_2 = set()
    for idx, line in enumerate(rdr):
        if idx <= 90000000:
            continue
        pfam_protein_set_2.add(line[0])
        if idx % 1000000 == 0:
            clear_output()
            print('row', idx)

row 181000000


In [6]:
with open('idr_annotations_no_pfam.csv') as f:
    with open('idr_annotations_pfam_proteins_2.csv', 'w') as fOut:
        for idx, line in enumerate(f):
            l = line.split(',')
            if l[0] in pfam_protein_set_2 and l[1][:-1] != '':
                fOut.write(line)
            if idx % 1000000 == 0:
                clear_output()
                print(f'Processing Row {idx}')

Processing Row 230000000


In [7]:
del pfam_protein_set_2

In [8]:
def parse_doms(dom_str):

    def sort_domains(interval_list):
        idx = sorted(range(len(interval_list)), key=lambda k: interval_list[k][0])
        return [interval_list[i] for i in idx]

    unsorted = [list(map(int, it[1].split(':'))) for it in (dom.split(';') for dom in dom_str.split('|'))]

    return sort_domains(unsorted)

In [9]:
def merge_sorted_intervals(sorted_intervals):
    merged_intervals = [sorted_intervals[0]]
    for intrvl in sorted_intervals[1:]:
        if intrvl[0] > merged_intervals[-1][1]:
            merged_intervals.append(intrvl)
        else:
            merged_intervals[-1][1] = max(merged_intervals[-1][1], intrvl[1])
    return merged_intervals

In [10]:
def intrvl_list_to_str(intrvl_list):
    output = ''
    for intrvl in intrvl_list:
        output += str(intrvl[0]) + ':' + str(intrvl[1]) + '|'
    return output[:-1]

In [11]:
with open('idr_annotations_pfam_proteins.csv') as f:
    with open('idr_annotations_pfam_proteins_merged.csv', 'w') as fOut:
        rdr = csv.reader(f)
        for idx, line in enumerate(rdr):
            fOut.write(line[0]+','+intrvl_list_to_str(merge_sorted_intervals(parse_doms(line[1][:-1])))+'\n')
            if idx % 1000000 == 0:
                clear_output()
                print(f'Processing Row {idx}')

Processing Row 17000000


In [12]:
with open('idr_annotations_pfam_proteins_2.csv') as f:
    with open('idr_annotations_pfam_proteins_2_merged.csv', 'w') as fOut:
        rdr = csv.reader(f)
        for idx, line in enumerate(rdr):
            fOut.write(line[0]+','+intrvl_list_to_str(merge_sorted_intervals(parse_doms(line[1][:-1])))+'\n')
            if idx % 1000000 == 0:
                clear_output()
                print(f'Processing Row {idx}')

Processing Row 22000000
